In [2]:
# Standard packages
import tempfile
import warnings
import urllib
import shutil
import os
# Less standard, but still pip- or conda-installable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
#import rasterio
import re
import rtree
import shapely
import pickle
import data_eng.az_proc as ap
import data_eng.form_calcs as fc

#from cartopy import crs
import collections
import cv2
import math
from glob import glob
from tqdm.notebook import tqdm_notebook

# Load Data

## Tile names and urls for all tiles apart of the dataset (complete array)

In [4]:
Z:\verified\verified_sets\verify_jaewon_poonacha_cleave_1\chips_positive_xml

(2610, 4)
unique formatted tile names (2540, 4)
unique standard tile names (2439, 4)


## Option 1. Multiple Tiles Cover Same Area With Different Years
Duplicates that cover the same area

- the area covers the same space
- Taken at different times or by different agencies
    - At different angles 
- tif have different sizes -> The the image chips are not identical 

In [12]:
u, c = np.unique(downloaded_tiles_info[:,3], return_counts=True)
dup = u[c > 1]
sum(np.isin(downloaded_tiles_info[:,3],dup))

99

In [ ]:
## Option 2. Multiple Annotaters were allocated the same tile

identify tiles that were allocated multiple tiles


In [41]:
u, i = np.unique(tile_img_annotation_annotator[:,[1,4,5,6,7]],axis=0, return_index=True)
tile_img_annotation_annotator = tile_img_annotation_annotator[i,:]

In [44]:
np.save("outputs/tile_img_annotation_annotator.npy", tile_img_annotation_annotator)

column_names = ["tile_name", "chip_name", "chip pathway", "xml annotation", 
                "annotator - draw","annotator - verify coverage",
                "annotator - verify quality", "annotator - verify classes"]

tile_img_annotation_annotator_df = pd.DataFrame(data = tile_img_annotation_annotator, 
                                               index = tile_img_annotation_annotator[:,1], 
                                               columns = column_names)

tile_img_annotation_annotator_df.to_csv('outputs/tile_img_annotation_annotator_df.csv')

In [57]:
#unique tiles and corresponding annotators
unique_tiles_annotators = np.unique(tile_img_annotation_annotator[:,[0,4]], axis=0)
print(len(unique_tiles_annotators))
unique_tiles, c = np.unique(unique_tiles_annotators[:,0], axis=0, return_counts = True)
print(len(unique_tiles))
duplicate_annotated_tiles = unique_tiles[c > 1]

2136
2029


array([], shape=(0, 8), dtype='<U200')

In [64]:
sum(np.isin(tile_img_annotation_annotator[:,0],duplicate_annotated_tiles))
possible_image_duplicates = tile_img_annotation_annotator[np.isin(tile_img_annotation_annotator[:,0],duplicate_annotated_tiles)]

In [82]:
len(np.unique(possible_image_duplicates[:,1]))

unique_duplicate_images, c = np.unique(possible_image_duplicates[:,1], axis=0, return_counts = True)
duplicate_annotated_images = unique_duplicate_images[c > 1]
duplicate_annotated_images = tile_img_annotation_annotator[np.isin(tile_img_annotation_annotator[:,1],duplicate_annotated_images)]

In [96]:
np.unique(duplicate_annotated_images[(duplicate_annotated_images[:,5]=='Cleave') & \
                           (duplicate_annotated_images[:,6]=='Sunny') & \
                           (duplicate_annotated_images[:,7]=='Niculescu')][:,0])

array([], dtype='<U200')

## Option 2. Overlaping state tiles
Some tiles are included that have the same coverage, same dates, but were duplicated because multiple states border AND the standard tiles are the same
identify tiles that were allocated multiple tiles

Note: Should be fixed with chip rename

In [ ]:
u, c = np.unique(tile_names_tile_urls_complete_array_unique_formatted_tile_names[:,2], return_counts = True)
dup = u[c > 1]

duplicate_standard_tile_names = tile_names_tile_urls_complete_array_unique_formatted_tile_names[np.isin(tile_names_tile_urls_complete_array_unique_formatted_tile_names[:,2],dup)]
pd.DataFrame(duplicate_standard_tile_names).to_csv("duplicate_standard_tile_names.csv")

In [ ]:
## Other Attempts

http://www.learningaboutelectronics.com/Articles/How-to-match-an-image-embedded-in-another-image-Python-OpenCV.php
https://www.quora.com/What-algorithms-can-detect-if-two-images-objects-are-similar-or-not
https://docs.opencv.org/2.4/modules/nonfree/doc/feature_detection.html?highlight=surf#feature-detection-and-description
https://docs.opencv.org/3.4/da/df5/tutorial_py_sift_intro.html
https://docs.opencv.org/3.4/dc/dc3/tutorial_py_matcher.html
https://docs.opencv.org/4.x/dc/dc3/tutorial_py_matcher.html
https://www.analyticsvidhya.com/blog/2019/08/3-techniques-extract-features-from-image-data-machine-learning-python/

Duplicate Sizes to identify duplicate images

In [ ]:
size = os.path.getsize('f:/file.txt')

In [26]:
gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
sift = cv2.SIFT_create()
kp = sift.detect(gray,None)
img=cv2.drawKeypoints(gray,kp,img)
#cv.imwrite('sift_keypoints.jpg',img)

In [ ]:
(score, diff) = compare_ssim(gray_image, gray_tile)
diff = (diff * 255).astype("uint8")

In [ ]:
import cv2
img1 = cv2.imread('/content/det1.jpg', 0)
img2 = cv2.imread('/content/88.jpg', 0)
orb = cv2.ORB_create(nfeatures=500)
kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1, des2)
matches = sorted(matches, key=lambda x: x.distance)
match_img = cv2.drawMatches(img1, kp1, img2, kp2, matches[:50], None)
cv2.imshow('original image', img1)
cv2.imshow('test image', img2)
cv2.imshow('Matches', match_img)
cv2.waitKey()

In [58]:
image1 = cv2.imread(os.path.join(parent_directory,
                "unverified_images\student_reviewed_unverified_images_set1\Alex\student_reviewed_images1\chips_positive",
                "al_60cm_2019_30088_m_3008808_sw_16_060_20191115_000126.jpg"),cv2.IMREAD_GRAYSCALE) # queryImage (img1)
image2 = cv2.imread(os.path.join(parent_directory,
                "unverified_images\student_reviewed_unverified_images_set1\Alex\student_reviewed_images1\chips_positive",
                "al_60cm_2019_30088_m_3008808_sw_16_060_20191115_000444.jpg"),cv2.IMREAD_GRAYSCALE) # queryImage (img1)
#tile = cv2.imread('box_in_scene.png',gray_image) # trainImage(tile2)

# Initiate SIFT detector
sift = cv2.SIFT_create()

# find the keypoints and descriptors with SIFT
kp_image1, des_image1 = sift.detectAndCompute(image1, None)
kp_image2, des_image2 = sift.detectAndCompute(image2, None)

In [59]:
# BFMatcher with default params
bf = cv2.BFMatcher()
matches = bf.knnMatch(des_image1, des_image2,k=2)
#bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des_image, des_tile)

matches = sorted(matches, key=lambda x: x.distance)
# Apply ratio test
good = []
for m,n in matches:
    if m.distance < 0.75*n.distance:
        good.append([m])
# cv.drawMatchesKnn expects list of lists as matches.
img3 = cv.drawMatchesKnn(des_image1, kp_image1, des_image2, kp_image1, good, None,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img3)
plt.show()

In [65]:
incorrect_labeled_chip_names_by_subfolder = pd.read_csv("incorrect_labeled_chip_names_by_subfolder.csv")
incorrect_labeled_chip_image_array = np.zeros((len(incorrect_labeled_chip_names_by_subfolder),512,512, 3), dtype='uint8')
for num in tqdm.tqdm(range(len(incorrect_labeled_chip_names_by_subfolder))):    
    labeled_chip_array = cv2.imread(incorrect_labeled_chip_names_by_subfolder["incorrect_chip_paths"][num])  #open image
    incorrect_labeled_chip_image_array[num,:,:,:] = labeled_chip_array


### convert to incorrect_labeled_chip_image_array to gray scale
gray_incorrect_labeled_chip_image_array = np.empty((len(incorrect_labeled_chip_image_array), 512, 512),dtype='uint8')

for num in range(len(incorrect_labeled_chip_image_array)):
    gray_incorrect_labeled_chip_image_array[num,:,:]  = cv2.cvtColor(incorrect_labeled_chip_image_array[num,:,:,:],
                                                                     cv2.COLOR_BGR2GRAY)
    
gray_incorrect_labeled_chip_image_unique_array, gray_incorrect_labeled_chip_image_unique_indices = np.unique(gray_incorrect_labeled_chip_image_array, axis = 0, return_index=True)

100%|██████████| 2690/2690 [01:20<00:00, 33.24it/s]


In [80]:
means = np.mean(gray_incorrect_labeled_chip_image_unique_array,axis=(1,2))

In [ ]:
# Weird things 

In [ ]:
chips_positive_path = "C:/chip_allocation/complete_dataset/chips_positive"
unique_chips_positive_path = "C:/chip_allocation/complete_dataset/unique_chips_positive"
dups_chips_positive_path = "C:/chip_allocation/complete_dataset/dups_chips_positive"

images, imgsr, imgsg, imgsb = fc.positive_images_to_array(chips_positive_path)
unique_imgsr, duplicate_imgsr =fc.unique_by_first_dimension(imgsr, images)
unique_imgsg, duplicate_imgsg = fc.unique_by_first_dimension(imgsg, images)
unique_imgsb, duplicate_imgsb = fc.unique_by_first_dimension(imgsb, images)
unique_images = fc.intersection_of_sets(unique_imgsr, unique_imgsg, unique_imgsb)
duplicate_images = fc.intersection_of_sets(duplicate_imgsr, duplicate_imgsg, duplicate_imgsb)   

In [ ]:
fc.directory_tile_names(os.path.join(parent_directory,"unverified_images/student_reviewed_unverified_images_set1/Qianyu/chips_positive"),
                       'Qianyu 1 tile names')

fc.directory_tile_names("C:/chip_allocation/complete_dataset/dups_chips_positive",
                        'dup tile names')

fc.identify_all_paths_to_duplicate_images(parent_directory, duplicate_images)

pd.DataFrame(fc.sorted_list_of_files(dups_chips_positive_path), columns = ['chip_names']).to_csv('dups_chips_positive_path.csv') 

In [70]:
for folder in folders:
    fc.remove_thumbs(folder)
    for jpg in os.listdir(folder):
        temp_jpg = os.path.splitext(jpg)[0].rsplit("_",1)[0]
        chip_number = os.path.splitext(jpg)[0].rsplit("_",1)[1]
        if (temp_jpg == "la_60cm_2019_29093_m_2909310_sw_15_060_20190726") & (jpg in duplicate_images):
            #pa_60cm_2019_39075_m_3907508_nw_18_060_20191015
            old_tile_path = os.path.join(folder, jpg)
            new_tile_path = os.path.join(folder, "pa_60cm_2019_39075_m_3907508_nw_18_060_20191015_" + chip_number+".jpg")
            #la_60cm_2019_29093_m_2909310_sw_15_060_20190726
            
            if not os.path.exists(new_tile_path): 
                os.rename(old_tile_path, new_tile_path)
            if os.path.exists(new_tile_path) and os.path.exists(old_tile_path): #If the new tile path already exists, delete the old tile path (if it still exists)
                shutil.move(old_tile_path,
                            "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//duplicate_move/chips_positive/"+"pa_60cm_2019_39075_m_3907508_nw_18_060_20191015_" + chip_number+".jpg")

In [5]:
folders = [  "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images/student_reviewed_unverified_images_set1/complete_dataset/chips_positive_xml",
             "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images/student_reviewed_unverified_images_set1/Qianyu/chips_positive_xml"]
for folder in folders:
    for xml in os.listdir(folder):
        temp_xml = os.path.splitext(xml)[0].rsplit("_",1)[0]
        print(temp_xml)
        chip_number = os.path.splitext(xml)[0].rsplit("_",1)[1]
        if (temp_xml == "la_60cm_2019_29093_m_2909310_sw_15_060_20190726") & (xml in duplicate_images):
            #pa_60cm_2019_39075_m_3907508_nw_18_060_20191015
            old_tile_path = os.path.join(folder, xml)
            new_tile_path = os.path.join(folder, "pa_60cm_2019_39075_m_3907508_nw_18_060_20191015_" + chip_number+".xml")
            #la_60cm_2019_29093_m_2909310_sw_15_060_20190726
            
            #if not os.path.exists(new_tile_path): 
                #os.rename(old_tile_path, new_tile_path)
            #if os.path.exists(new_tile_path) and os.path.exists(old_tile_path): #If the new tile path already exists, delete the old tile path (if it still exists)
                #shutil.move(old_tile_path_xml,
                #            "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//duplicate_move/chips_positive_xml/"+"pa_60cm_2019_39075_m_3907508_nw_18_060_20191015_" + chip_number+".xml")

co_60cm_2019_39104_m_3910410_se_13_060_20190912


NameError: name 'duplicate_images' is not defined